<a href="https://colab.research.google.com/github/TomAmster/Data-Science/blob/master/EDA_Explaratory_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np

# **Explaratory Data Analysis**

## Loading the datasets


In [0]:
drive_dir = '/content/drive/Shared drives/DS Workshop/data/'
train_path = drive_dir + 'train.csv'
train = pd.read_csv(train_path)
bids_path = drive_dir + 'bids.csv/bids.csv'
bids = pd.read_csv(bids_path)
test_path = drive_dir + 'test.csv'
test = pd.read_csv(test_path)

### Merging Datasets- Train & Test with the Bids

In [0]:
bidders_df = pd.concat([train, test])
merged_bids = pd.merge(bidders_df, bids, on='bidder_id', how='left')
train_bids = pd.merge(train, bids, on='bidder_id', how='left')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


## General Dataset Information

In [0]:
print(f"total row in bid:          {len(merged_bids)}")
print(f"total bids in bid:         {len(merged_bids['bid_id'].unique())}")
print(f"total bidder in bids:      {len(merged_bids['bidder_id'].unique())}")
print(f"total payment in bids:     {len(merged_bids['payment_account'].unique())}")
print(f"total address in bids:     {len(merged_bids['address'].unique())}")
print(f"total auction in bids:     {len(merged_bids['auction'].unique())}")
print(f"total merchandise in bids: {len(merged_bids['merchandise'].unique())}")
print(f"total device in bids:      {len(merged_bids['device'].unique())}")
print(f"total country in bids:     {len(merged_bids['country'].unique())}")
print(f"total ip in bids:          {len(merged_bids['ip'].unique())}")
print(f"total url in bids:         {len(merged_bids['url'].unique())}")

total row in bid:          7656433
total bids in bid:         7656335
total bidder in bids:      6713
total payment in bids:     6713
total address in bids:     6713
total auction in bids:     15052
total merchandise in bids: 11
total device in bids:      7352
total country in bids:     200
total ip in bids:          2303992
total url in bids:         1786352


In [0]:
print(
    f"total bids made by robots in data set: {len(bid[bid['outcome'] == 1])}")
print(
    f"total bids made by human in data set:  {len(bid[bid['outcome'] == 0])}")
print("the ratio of made by robot vs. bids made by human in data set: 3:20")

total bids made by robots in data set: 412416
total bids made by human in data set:  2658837
the ratio of made by robot vs. bids made by human in data set: 3:20


## Statistics Analysis - 
Checking Difference in pararmeters between robots and humans

### Creating Count Statistics for the pararmeters

In [0]:
bids_count = bids.groupby("bidder_id")["bid_id","device","country","ip","merchandise","auction","url"].nunique()
bids_count.reset_index(inplace=True)
bids_count = bids_count.add_prefix("count_")
bids_count.rename(columns={"count_bidder_id":"bidder_id"},inplace=True)
train_counts = pd.merge(train,bids_count,on="bidder_id",how="left")

In [0]:
train_counts.head()

,bidder_id,payment_account,address,outcome,count_bid_id,count_device,count_country,count_ip,count_merchandise,count_auction,count_url
0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,14.0,6.0,20.0,1.0,18.0,1.0
1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,2.0,1.0,3.0,1.0,1.0,2.0
2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,2.0,1.0,4.0,1.0,4.0,2.0
3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,53.0,2.0,123.0,1.0,23.0,91.0


### Creating Count Statistics Per Parameter Per Auction

In [0]:
train_counts['average_bids_per_auction'] = train_counts.apply(lambda row:row['count_bid_id']/row['count_auction'] if row['count_auction']>0 else 0, axis=1) 
train_counts['average_bids_per_url'] = train_counts.apply(lambda row:row['count_bid_id']/row['count_url'] if row['count_url']>0 else 0, axis=1) 
train_counts['average_bids_per_device'] = train_counts.apply(lambda row:row['count_bid_id']/row['count_device'] if row['count_device']>0 else 0, axis=1) 
train_counts['average_bids_per_country'] = train_counts.apply(lambda row:row['count_bid_id']/row['count_country'] if row['count_country']>0 else 0, axis=1) 

In [0]:
train_counts.head()

,bidder_id,payment_account,address,outcome,count_bid_id,count_device,count_country,count_ip,count_merchandise,count_auction,count_url,average_bids_per_auction,average_bids_per_url,average_bids_per_device,average_bids_per_country
0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,24.0,14.0,6.0,20.0,1.0,18.0,1.0,1.333333,24.000000,1.714286,4.0
1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,3.0,2.0,1.0,3.0,1.0,1.0,2.0,3.000000,1.500000,1.500000,3.0
2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,2.0,1.0,4.0,1.0,4.0,2.0,1.000000,2.000000,2.000000,4.0
3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.0
4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,155.0,53.0,2.0,123.0,1.0,23.0,91.0,6.739130,1.703297,2.924528,77.5


In [0]:
human_data = train_counts[train_counts['outcome'] == 0]
robot_data = train_counts[train_counts['outcome'] == 1]

In [0]:
print(f"average bids per robot: %.0f" % robot_data.count_bid_id.mean() )
print(f"average bids per human:  %.0f" % human_data.count_bid_id.mean())
print(f"average bids per auction by robot: %.0f" % robot_data.average_bids_per_auction.mean())
print(f"average bids per auction by human:  %.0f" % human_data.average_bids_per_auction.mean())
print(f"average number of devices by robot: %.0f" % robot_data.count_device.mean())
print(f"average number of devices by human:  %.0f" % human_data.count_device.mean())
print(f"average number of IPs by robot: %.0f" % robot_data.count_ip.mean())
print(f"average number of IPs by human:  %.0f" % human_data.count_ip.mean())
print(f"average number of URLs by robot: %.0f" % robot_data.count_url.mean())
print(f"average number of URLs by human:  %.0f" % human_data.count_url.mean())

average bids per robot: 4004
average bids per human:  1414
average bids per auction by robot: 23
average bids per auction by human:  6
average number of devices by robot: 164
average number of devices by human:  74
average number of IPs by robot: 2388
average number of IPs by human:  581
average number of URLs by robot: 545
average number of URLs by human:  335
